# Problem Sheet: Convolution and Deblurring

## Objectives
- Implement 1D/2D convolution with different boundary conditions
- Understand kernel effects on signals/images
- Implement deblurring via inverse Fourier transform and observe effect of different noise

---

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2, fftshift, ifftshift
from skimage import data

# Exercise 1: 1D Convolution
For discrete signals $f$ and $g$, convolution $(f * g)[n]$ is defined as:

$$
(f * g)[n] = \sum_{m=-\infty}^{\infty} f[m] \cdot g[n - m]
$$

## 1. Zero-Padded Convolution
### Definition
For a signal `x[n]` of length `N` and kernel `h[m]` of length `M`:

$$
(x *_{\text{zero}} h)[n] = \sum_{m=0}^{M-1} h[m] \cdot \tilde{x}[n - m]
$$

where the padded signal $\tilde{x}[k]$ is:

$$
\tilde{x}[k] = 
\begin{cases} 
x[k] & \text{if } 0 \leq k < N \\
0 & \text{otherwise}
\end{cases}
$$

### Properties
- **Output length**: $N$ (same-mode; we implement this one) or $N + M - 1$ (full).
- **Edge behavior**: Introduces artificial zeros at boundaries  
- **Best for**: Non-periodic signals (e.g., natural images, sensor data)

## 2. Cyclic (Periodic) Convolution

### Definition
For a discrete signal $x[n]$ of length $N$ and kernel $h[m]$ of length $M$:

$$
(x *_{\text{cyclic}} h)[n] = \sum_{m=0}^{M-1} h[m] \cdot x[(n - m) \mod N]
$$

where:
- $x[k \mod N]$ enforces periodicity (e.g., $x[-1] = x[N-1]$)
- Output length is exactly $N$ (same as input)

## Key Properties
| Property          | Description |
|-------------------|-------------|
| **Periodicity**   | Treats signal as infinitely repeating: $x[n] = x[n \mod N]$ |
| **Edge Handling** | Wraps around boundaries (no discontinuities) |
| **FFT Relation**  | Equivalent to pointwise multiplication in Fourier domain: $\mathcal{F}(x *_{\text{cyclic}} h) = \mathcal{F}(x) \cdot \mathcal{F}(h)$ |
| **Artifacts**     | May create false correlations between opposite ends of the signal |



## Task
- Implement the `convolve_1d()` function that suppords 1D convolution with different boundary conditions
- Plot the convolution for a given rectangular pulse with a Gaussian Kernel with zero-padded vs. cyclic boundary condition.
- For what kind of signal would the boundary condition affect the result of the convolution?

Below you are given some starter code.



In [ ]:
def convolve_1d(signal, kernel, mode='zero-pad'):
    """
    Args:
        signal: 1D input array
        kernel: 1D convolution kernel
        mode: 'zero-pad' or 'cyclic'
    
    Returns:
        Convolved signal (same length as input)
    """
    # Your implementation here
    # Hint: use np.pad with mode 'wrap' or 'constant'
    
# Signal: rectangular pulse
signal = np.zeros(100)
signal[20:60] = 1.0 

# Kernel setup
kernel_size = 15
sigma = 3.0
positions = np.linspace(-kernel_size//2, kernel_size//2, kernel_size)
kernel = np.exp(-positions**2 / (2 * sigma**2))
kernel /= np.sum(kernel)

# Plotting 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot original signal
ax1.plot(signal, linewidth=2)
ax1.set_title("Original Signal")
ax1.set_xlabel("Sample index")
ax1.set_ylabel("Amplitude")

# Plot kernel
ax2.stem(positions, kernel, linefmt='C1-', markerfmt='C1o')
ax2.set_title("Gaussian Kernel")

# Exercise 2: 2D Convolution with Different Kernels
For images $I$ and kernel $K$:

$$
(I * K)[x,y] = \sum_{i=-\infty}^{\infty} \sum_{j=-\infty}^{\infty} I[i,j] \cdot K[x-i, y-j]
$$

## Tasks
- Implement `convolve_2d()` with zero-padded boundaries.
- Compute convolution of the Camera-Man-Image with the following 2d kernels:
    - (a) Gaussian Kernel
    - (b) X-Derivative Kernel
    - (c) Y-derivative
    - (d) Laplacian Kernel
- Visualize the kernels under the convolutions
- Understand effects of different kernel types. What does the Gaussian Kernel do? What does the Laplacian kernel do? 

Below is a starter code.

In [ ]:
## Part 1: Implement 2D Convolution
def convolve_2d(image, kernel):
    """
    Perform 2D convolution with zero-padding.
    
    Args:
        image: 2D input array (grayscale image)
        kernel: 2D convolution kernel (smaller than image)
    
    Returns:
        Convolved image (same size as input)
    """
    # TO DO: Implement the convolution operation
    # Hint: You'll need to:
    # 1. Pad the image with periodic boundaries
    # 2. Flip the kernel (both dimensions)
    # 3. Apply the kernel to each pixel neighborhood
    return np.zeros_like(image)  # Replace this with your implementation

## Part 2: Implement Kernels
def gaussian_kernel(size=5, sigma=1.0):
    """
    Generate 2D Gaussian kernel.
    size: kernel dimensions (size x size)
    sigma: controls blur strength (larger = more blur)
    """
    # TO DO: Create a Gaussian kernel
    # Hint: Use np.exp and normalization
    return np.ones((size,size))/(size*size)  # Replace with Gaussian kernel

def derivative_x_kernel():
    """Sobel operator for x-direction edges"""
    # Returns 3x3 x-derivative kernel
    return np.array([[1, 0, -1],
                    [2, 0, -2],
                    [1, 0, -1]]) / 8  

def derivative_y_kernel():
    """Sobel operator for y-direction edges"""
    # TO DO: Return 3x3 y-derivative kernel analogous to derivative_x_kernel
    return np.ones((3,3))  # Replace with correct kernel

def laplacian_kernel():
    """Discrete Laplacian operator"""
    # TO DO: Return 3x3 Laplacian kernel
    # Hint: think of f′′(x)≈f(x+1)+f(x−1)−2f(x)
    return np.ones((3,3))  # Replace with correct kernel

## Part 3: Load Image and Prepare Kernels
image = data.camera().astype(float)/255  # Normalize to [0,1]

# Experiment with different blur amounts!
sigma = 4.5  # Try values between 0.5 and 5.0

kernels = {
    'Gaussian': gaussian_kernel(15, sigma),  # Large kernel for visible blur
    'X-Derivative': derivative_x_kernel(),
    'Y-Derivative': derivative_y_kernel(),
    'Laplacian': laplacian_kernel()
}

## Visualization Code (Complete)
plt.figure(figsize=(14, 6))

# First row: Original + Convolved Images
for i, (name, kernel) in enumerate(kernels.items(), 1):
    plt.subplot(2, 4, i)
    if i == 1:
        plt.imshow(image, cmap='gray')
        plt.title("Original")
    else:
        result = convolve_2d(image, kernel)
        if name != 'Gaussian':
            result = (result - result.min()) / (result.max() - result.min())
        plt.imshow(result, cmap='gray')
        plt.title(name)
    plt.axis('off')

# Second row: Kernel Visualizations
plt.subplot(2, 4, 5)  # Empty
plt.axis('off')

for i, (name, kernel) in enumerate(kernels.items(), 6):
    plt.subplot(2, 4, i)
    cmap = 'hot' if name == 'Gaussian' else 'coolwarm'
    vmax = None if name == 'Gaussian' else max(abs(kernel.min()), kernel.max())
    plt.imshow(kernel, cmap=cmap, vmin=-vmax, vmax=vmax)
    plt.colorbar()
    plt.title(f"{name} Kernel")
    plt.axis('off')

plt.tight_layout()
plt.show()

# Exercise 3: Image Deblurring via Fourier Methods

## Background

We consider the image formation model:

$$ B = I * K + \eta $$

where:
- $I$ is the true image (unknown)
- $K$ is the blur kernel (known Point Spread Function)
- $\eta$ is additive noise
- $B$ is the observed blurred image

For periodic boundary conditions, the convolution theorem gives:

$ \mathcal{F}\{I*K\} = \mathcal{F}\{I\} \cdot \mathcal{F}\{K\}$

where $\mathcal{F}$ denotes the 2D Fourier transform. If the noise is zero, we can deblur the image as

$I = \mathcal{F}^{-1}\{\frac{\mathcal{F}\{I*K\}}{\mathcal{F}\{K\}}\}$

## Task

Implement the Fourier deconvolution approach and apply it to a blurred image with (a) zero-noise, (b) Gaussian-noise, (c) scaled high-frequency sine-wave-noise. What do you observe? Why does the deblurring fails  even with relatively small noise?

In [ ]:
# Starter code

def gaussian_kernel(size=25, sigma=4.5):
    """Generate 2D Gaussian kernel"""
    # STUDENT TODO: Implement Gaussian kernel generation
    # Hint: Use np.linspace and np.meshgrid
    return kernel

def blur_image_fourier(image, kernel):
    """Blur image using Fourier convolution theorem"""
    # STUDENT TODO: 
    # 1. Pad kernel to match image size
    # 2. Compute FFTs
    # 3. Multiply in frequency domain
    # 4. Transform back
    return blurred

def deblur_image_fourier(blurred, kernel, epsilon=1e-9):
    """Deblur image using inverse filtering"""
    # STUDENT TODO:
    # 1. Pad kernel to match image size
    # 2. Compute FFTs
    # 3. Apply inverse filter adding an epsilon in denominator for stability 
    # 4. Transform back
    return deblurred

# Main processing pipeline
def main():
    # Load and prepare image
    image = img_as_float(data.camera())
    
    # STUDENT TODO: Create kernel
    kernel = gaussian_kernel(size=25, sigma=4.5)
    
    # Blur the image
    blurred = blur_image_fourier(image, kernel)
    
    # Create different noise cases
    # STUDENT TODO: Add Gaussian noise
    noise_gaussian = np.random.normal(0, 0.02, image.shape)
    
    # STUDENT TODO: Add high-frequency sine noise
    x = np.linspace(0, 20*np.pi, image.shape[1])
    y = np.linspace(0, 20*np.pi, image.shape[0])
    X, Y = np.meshgrid(x, y)
    noise_sine = 0.1 * (np.sin(15*X) + np.sin(15*Y))
    
    # Create noisy versions
    blurred_gauss = blurred + noise_gaussian
    blurred_sine = blurred + noise_sine
    
    # Deblur all cases
    epsilon = 1e-9
    deblurred_clean = deblur_image_fourier(blurred, kernel, epsilon)
    deblurred_gauss = deblur_image_fourier(blurred_gauss, kernel, epsilon)
    deblurred_sine = deblur_image_fourier(blurred_sine, kernel, epsilon)
    
    # Plot results
    plt.figure(figsize=(15, 10))
    
    # Top row: Noisy images
    plt.subplot(2, 3, 1)
    plt.imshow(blurred, cmap='gray')
    plt.title('Blurred (No Noise)')
    plt.axis('off')
    
    plt.subplot(2, 3, 2)
    plt.imshow(blurred_gauss, cmap='gray')
    plt.title('+ Gaussian Noise')
    plt.axis('off')
    
    plt.subplot(2, 3, 3)
    plt.imshow(blurred_sine, cmap='gray')
    plt.title('+ HF Sine Noise')
    plt.axis('off')
    
    # Bottom row: Deblurred results
    plt.subplot(2, 3, 4)
    plt.imshow(deblurred_clean, cmap='gray', vmin=0, vmax=1)
    plt.title('Deblurred (Clean)')
    plt.axis('off')
    
    plt.subplot(2, 3, 5)
    plt.imshow(deblurred_gauss, cmap='gray', vmin=0, vmax=1)
    plt.title('Deblurred (Gaussian)')
    plt.axis('off')
    
    plt.subplot(2, 3, 6)
    plt.imshow(deblurred_sine, cmap='gray', vmin=0, vmax=1)
    plt.title('Deblurred (Sine)')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()